# Lab Notebook for Matthew Wright

# Write some Python
1. Open ```batch_2.fst_1-2.tsv``` from your fst directory.
2. Read each line and remove the newlines.
3. Split each line by tabs and save the resulting elements in an array.
4. Sum the F<sub>ST</sub> values from the file and calculate the average F<sub>ST</sub> and print it out
6. Print a unique list of chromosomes contained in the file.

In [49]:
with open("../2017-ica6-louislamont/fst/batch_2.fst_1-2.tsv", "r") as fh:
    i=0
    fst=0
    chroms=[]
    next(fh)
    for line in fh:
        larray=line.strip("\n").split("\t")
        fst+=float(larray[8])
        i+=1
        if "group" in larray[4]:
            try:
                chroms.index(larray[4])
            except ValueError:
                chroms.append(larray[4])
    
print("Avg Fst:", (fst/(i)))
print(chroms)

Avg Fst: -0.0006395945316634855
['groupI', 'groupII', 'groupIII', 'groupIV', 'groupIX', 'groupV', 'groupVI', 'groupVII', 'groupVIII', 'groupX', 'groupXI', 'groupXII', 'groupXIII', 'groupXIV', 'groupXIX', 'groupXV', 'groupXVI', 'groupXVII', 'groupXVIII', 'groupXX', 'groupXXI']


# 7/6/2017
# Blast

5 types:
BLASTn - nucleotide vs nucleotide search
BLASTt - protein v protein search
BLASTx - protein database v translated nucleotide query
tBLASTn - translated nucleotide database v protein query
tBLASTx - translated nucleotide database v translated nucleotide database

4 steps:
1. Seeding:
    1. Break query into k-mers (11 for nucleotides, 3 for amino acids)
    2. Has a threshold T that allows for mismatches between query and database
    3. BLAST scans the database for occurences of k-mers derived from the query sequence
    4. Requires two word matches for amino acids, one word is sufficient for nucleotides
2. Non-gapped extension:
    1. Uses a score threshold to stop extensions ("Drop-off score")
    2. Tracks score alignment since last maximum
3. Gapped extension
    1. Smith-Waterman alignment, dynamic programming
4. Evaluation
    1. Score is a combination of matches, mismatch penalties, gap opening penalties, and gap extension penalties
    2. HSPs - High-scoring pairs. HSPs are clustered if they are consistent
    3. E-value: the number of alignments expected by chance during a sequence database search
        $$E=kmne^{-\lambda S}$$
        m: length of query;
        n: length of database;
        k: constant;
        $\lambda S$: normalized score
    4. a function of the search space (m * n), the normalized score, and a constant k

# BLAT - The BLAST-Like Alignment Tool

* Created by James Kent in order to assist annotating and assembling the human genome as it was being sequenced.
* To assess human genome quality, Kent was repeatedly aligning human and mouse ESTs (gene transcripts) to the growing genome


## Basic Algorithm
1. Index the database using all non-overlapping k-mers with their positions
2. Look up every k-mer in the query in the database
    * How would you allow mismatches in query k-mers? Trick question: shrink k-mers so there are no mismatches
3. nucleotide BLAT requires two perfect matching 11-mers
4. Nearby 11-mers are clumped if they fall consistently on the diagonal
5. k-mers are extended one letter at a time as long as perfect matches occur, k-mers are merged together
6. Smaller k-mers are generated from gapped areass and the extension occurs recursively
7. Finally, special rules are applied to fill in the gapped areas

# BLAT vs. BLAST
* BLAST is most useful in searching for distantly related homologs
    * BLAST is optimized for gene-size lengths of sequence
* BLAT is more useful for finding matches between much more closely related sequence, particularly nucleotides
    * BLAT is particuarly useful for transcriptome alignments (mRNA) and nucleotide contigs from genome assemblies
* BLAT used to have a large performance advantage over BLAST, but not as important now

# K-mers

k-mers in a sequence: $seqlen - k+1$

* A k-mer represnts a unique sequence of the genome
* k-mers occur in proportion to sequencing depth
* sequencing error is a major source of rare k-mers

* The concept of trusted and untrusted k-mers
    * sequencing errors introduce rare k-mers, you can correct untrusted k-mers if there are high coverage k-mers at that spot
    
### Digital Normalization

1. Tally the occurence of each k-mer in the read.
2. Look up the number of occurences of each k-mer
3. Calculate the median k-mer coverage for the read
4. Drop the read if the coverage > threshold